<a href="https://colab.research.google.com/github/vishalveerareddy/VGG16/blob/master/CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import _pickle as pickle
import numpy as np

import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import plot_model


## Parameters
num_classes = 10
num_epochs = 20
num_predictions = 10
save_dir = os.path.join(os.getcwd(), 'model')
model_name = 'cifar10_deepcnn.h5'

## Hyper-parameters
batch_size = 32




In [3]:
if __name__ == "__main__":
    ## Getting data
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    # Convert class vectors to binary class matrices (one-hot vectors)
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)

    ## Load model if existed
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    model_path = os.path.join(save_dir, model_name)

    if os.path.exists(model_path):
        print("Load pre-trained model %s" % model_path)
        model = load_model(model_path)
    else:
        ## else, build model
        model = Sequential()
        model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
        model.add(Activation('relu'))
        model.add(Conv2D(32, (3, 3)))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        model.add(Flatten())
        model.add(Dense(512))
        model.add(Activation('relu'))
        model.add(Dropout(0.5))
        model.add(Dense(num_classes))
        model.add(Activation('softmax'))

        # optimizer
        opt = keras.optimizers.rmsprop(lr=1e-4, decay=1e-6)

        model.compile(loss='categorical_crossentropy',
                      optimizer=opt,
                      metrics=['accuracy'])

        x_train = x_train.astype('float32')
        x_test = x_test.astype('float32')
        x_train /= 255
        y_train /= 255

        ## Data augumentation
        datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False  # randomly flip images
        )
        datagen.fit(x_train)

        ## Let's train
        model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                            steps_per_epoch=x_train.shape[0] // batch_size,
                            epochs=num_epochs,
                            validation_data=(x_test, y_test))

        ## Save model
        model.save(model_path)
        plot_model(model, to_file='cifar10_deepcnn.jpg')
        print("Save model at %s" % model_path)

    ## Evaluate
    # load label names to use in prediction results
    label_list_path = 'datasets/cifar-10-batches-py/batches.meta'

    keras_dir = os.path.expanduser(os.path.join('~', '.keras'))
    datadir_base = os.path.expanduser(keras_dir)
    if not os.access(datadir_base, os.W_OK):
        datadir_base = os.spath.join('/tmp', '.keras')
    label_list_path = os.path.join(datadir_base, label_list_path)

    with open(label_list_path, mode='rb') as f:
        labels = pickle.load(f)
    print("Load label names %s" % label_list_path)

    # Evaluate with test dataset and share same prediction results
    evaluation = model.evaluate_generator(datagen.flow(x_test, y_test, batch_size=batch_size),
                                          steps=x_test.shape[0] // batch_size)
    print('Model accuracy = %.2f' % evaluation[1])

    predict_gen = model.predict_generator(datagen.flow(x_test, y_test, batch_size=batch_size),
                                          steps=x_test.shape[0] // batch_size)
    for predict_index, predicted_y in enumerate(predict_gen):
        actual_label = labels['label_names'][np.argmax(y_test[predict_index])]
        predicted_label = labels['label_names'][np.argmax(predicted_y)]
        print('Actual label = %s vs. Predicted label = %s' % (actual_label, predicted_label))

        if predict_index == num_predictions:
            break

170500096/170498071 [==============================] - 2s 0us/step


W0803 16:34:51.181890 139622331996032 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0803 16:34:51.533830 139622331996032 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0803 16:34:51.542321 139622331996032 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0803 16:34:51.606934 139622331996032 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0803 16:34:51.611816 139622331996032 deprecation_wrapp

Epoch 1/20
1562/1562 [==============================] - 249s 160ms/step - loss: 0.0068 - acc: 0.3806 - val_loss: 9.7378 - val_acc: 0.3878
Epoch 2/20
1562/1562 [==============================] - 247s 158ms/step - loss: 0.0059 - acc: 0.4634 - val_loss: 9.8293 - val_acc: 0.3834
Epoch 3/20
1562/1562 [==============================] - 247s 158ms/step - loss: 0.0055 - acc: 0.4992 - val_loss: 8.9363 - val_acc: 0.4394
Epoch 4/20
1562/1562 [==============================] - 246s 158ms/step - loss: 0.0052 - acc: 0.5263 - val_loss: 9.5695 - val_acc: 0.4012
Epoch 5/20
1562/1562 [==============================] - 246s 157ms/step - loss: 0.0050 - acc: 0.5466 - val_loss: 9.2787 - val_acc: 0.4187
Epoch 6/20
1562/1562 [==============================] - 243s 156ms/step - loss: 0.0049 - acc: 0.5620 - val_loss: 9.8351 - val_acc: 0.3846
Epoch 7/20
1562/1562 [==============================] - 246s 157ms/step - loss: 0.0047 - acc: 0.5765 - val_loss: 9.0726 - val_acc: 0.4321
Epoch 8/20
1562/1562 [============

In [4]:
model.evaluate(x_test,y_test)

10000/10000 [==============================] - 10s 974us/step


[7.443526116943359, 0.5328]